# Neural Network v2 - Mit Early Stopping & Regularisierung
## Overfitting reduzieren durch intelligentes Training

In dieser Version implementieren wir:
- **Early Stopping**: Stoppt Training automatisch, wenn Validation Loss nicht mehr besser wird
- **Dropout**: Regularisierung durch zufälliges Deaktivieren von Neuronen
- **L2 Regularization (Weight Decay)**: Bestrafung großer Gewichte
- **Learning Rate Scheduling**: Anpassung der Learning Rate während des Trainings

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import warnings
warnings.filterwarnings('ignore')

# Setze Random Seeds
np.random.seed(42)
torch.manual_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print(f"PyTorch Version: {torch.__version__}")

Using device: cpu
PyTorch Version: 2.9.1


## 1. Daten laden (100 Samples)

In [2]:
# Lade Datensatz
BASE_DIR = Path.cwd().parent
DATA_PATH = BASE_DIR / "housing.csv"

df_full = pd.read_csv(DATA_PATH)
print(f"Kompletter Datensatz: {len(df_full)} Zeilen")

# Reduziere auf 100 Samples (stratifiziert)
df_full['price_quartile'] = pd.qcut(df_full['median_house_value'], q=4, labels=False)
df_small = df_full.groupby('price_quartile', group_keys=False).apply(
    lambda x: x.sample(n=25, random_state=42)
).drop('price_quartile', axis=1).reset_index(drop=True)

print(f"Reduzierter Datensatz: {len(df_small)} Zeilen")

# Feature Engineering
def engineer_features(df):
    df = df.copy()
    df['households'] = df['households'].replace(0, np.nan)
    df['total_rooms'] = df['total_rooms'].replace(0, np.nan)
    df['rooms_per_household'] = df['total_rooms'] / df['households']
    df['bedrooms_per_room'] = df['total_bedrooms'] / df['total_rooms']
    df['population_per_household'] = df['population'] / df['households']
    return df

df_small = engineer_features(df_small)
df_small = pd.get_dummies(df_small, columns=['ocean_proximity'], drop_first=False)

# Train-Test Split
X = df_small.drop('median_house_value', axis=1)
y = df_small['median_house_value']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nTraining: {len(X_train)} | Test: {len(X_test)}")

# Preprocessing
imputer = SimpleImputer(strategy='median')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1)).flatten()
y_test_scaled = y_scaler.transform(y_test.values.reshape(-1, 1)).flatten()

# PyTorch Tensoren
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.FloatTensor(y_train_scaled).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.FloatTensor(y_test_scaled).to(device)

print(f"Features: {X_train_tensor.shape[1]}")

Kompletter Datensatz: 20640 Zeilen
Reduzierter Datensatz: 100 Zeilen

Training: 80 | Test: 20
Features: 15


## 2. Modelle MIT Regularisierung (Dropout)

Dropout deaktiviert zufällig Neuronen während des Trainings, um Overfitting zu verhindern.

In [3]:
class RegularizedSmallNet(nn.Module):
    """Small Model mit Dropout"""
    def __init__(self, input_dim, dropout_rate=0.2):
        super(RegularizedSmallNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 32)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(32, 16)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.fc3 = nn.Linear(16, 1)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

class RegularizedMediumNet(nn.Module):
    """Medium Model mit Dropout"""
    def __init__(self, input_dim, dropout_rate=0.3):
        super(RegularizedMediumNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(64, 32)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.fc3 = nn.Linear(32, 16)
        self.dropout3 = nn.Dropout(dropout_rate)
        self.fc4 = nn.Linear(16, 1)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.relu(self.fc3(x))
        x = self.dropout3(x)
        x = self.fc4(x)
        return x

class RegularizedLargeNet(nn.Module):
    """Large Model mit Dropout"""
    def __init__(self, input_dim, dropout_rate=0.4):
        super(RegularizedLargeNet, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(128, 64)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.fc3 = nn.Linear(64, 32)
        self.dropout3 = nn.Dropout(dropout_rate)
        self.fc4 = nn.Linear(32, 16)
        self.dropout4 = nn.Dropout(dropout_rate)
        self.fc5 = nn.Linear(16, 1)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout1(x)
        x = self.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.relu(self.fc3(x))
        x = self.dropout3(x)
        x = self.relu(self.fc4(x))
        x = self.dropout4(x)
        x = self.fc5(x)
        return x

input_dim = X_train_tensor.shape[1]

print("Modelle MIT Regularisierung:")
print("=" * 60)
print(f"Small Net  - Dropout: 20%")
print(f"Medium Net - Dropout: 30%")
print(f"Large Net  - Dropout: 40%")

Modelle MIT Regularisierung:
Small Net  - Dropout: 20%
Medium Net - Dropout: 30%
Large Net  - Dropout: 40%


## 3. Early Stopping Klasse

Stoppt das Training automatisch, wenn der Validation Loss nicht mehr sinkt.

In [4]:
class EarlyStopping:
    """Early Stopping um Training zu stoppen wenn Validation Loss nicht mehr sinkt"""
    
    def __init__(self, patience=20, min_delta=0.001, verbose=True):
        """
        Args:
            patience: Wie viele Epochen ohne Verbesserung abgewartet werden
            min_delta: Minimale Änderung, die als Verbesserung gilt
            verbose: Ausgabe von Meldungen
        """
        self.patience = patience
        self.min_delta = min_delta
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.best_epoch = 0
        
    def __call__(self, val_loss, epoch):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_epoch = epoch
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.verbose and self.counter % 5 == 0:
                print(f"    Early Stopping Counter: {self.counter}/{self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
                if self.verbose:
                    print(f"\n    ⚠️  Early Stopping bei Epoch {epoch}!")
                    print(f"    Bester Validation Loss bei Epoch {self.best_epoch}: {self.best_loss:.4f}")
        else:
            self.best_loss = val_loss
            self.best_epoch = epoch
            self.counter = 0
            
    def reset(self):
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.best_epoch = 0

print("Early Stopping Klasse erstellt!")
print("Stoppt Training wenn Validation Loss 20 Epochen nicht besser wird.")

Early Stopping Klasse erstellt!
Stoppt Training wenn Validation Loss 20 Epochen nicht besser wird.


## 4. Training mit Early Stopping & Weight Decay (L2)

In [5]:
def train_model_with_early_stopping(
    model, X_train, y_train, X_test, y_test, 
    max_epochs=500, 
    batch_size=16, 
    lr=0.001,
    weight_decay=0.01,  # L2 Regularization
    patience=20
):
    """Training mit Early Stopping und L2 Regularization"""
    
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    # Learning Rate Scheduler: Reduziert LR wenn kein Fortschritt
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=10, verbose=False
    )
    
    # DataLoader
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    # Early Stopping
    early_stopping = EarlyStopping(patience=patience, verbose=True)
    
    history = {'train_loss': [], 'val_loss': [], 'lr': []}
    
    for epoch in range(max_epochs):
        # Training
        model.train()
        train_losses = []
        
        for batch_X, batch_y in train_loader:
            outputs = model(batch_X).squeeze()
            loss = criterion(outputs, batch_y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_losses.append(loss.item())
        
        avg_train_loss = np.mean(train_losses)
        
        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_test).squeeze()
            val_loss = criterion(val_outputs, y_test).item()
        
        # Learning Rate anpassen
        scheduler.step(val_loss)
        current_lr = optimizer.param_groups[0]['lr']
        
        history['train_loss'].append(avg_train_loss)
        history['val_loss'].append(val_loss)
        history['lr'].append(current_lr)
        
        # Ausgabe alle 50 Epochen
        if (epoch + 1) % 50 == 0:
            print(f"Epoch [{epoch+1}/{max_epochs}] - Train: {avg_train_loss:.4f}, Val: {val_loss:.4f}, LR: {current_lr:.6f}")
        
        # Early Stopping Check
        early_stopping(val_loss, epoch + 1)
        if early_stopping.early_stop:
            break
    
    final_epoch = epoch + 1
    print(f"\n✓ Training beendet nach {final_epoch} Epochen")
    
    return history, final_epoch

print("Training-Funktion mit Early Stopping & L2 Regularization bereit!")

Training-Funktion mit Early Stopping & L2 Regularization bereit!


## 5. Trainiere regularisierte Modelle

In [6]:
# Hyperparameter
MAX_EPOCHS = 500
BATCH_SIZE = 16
LEARNING_RATE = 0.001
WEIGHT_DECAY = 0.01  # L2 Regularization
PATIENCE = 20  # Early Stopping Patience

# Trainiere Small Model
print("=" * 70)
print("TRAINING: Small Model (mit Dropout 20% + L2 + Early Stopping)")
print("=" * 70)
model_small_reg = RegularizedSmallNet(input_dim, dropout_rate=0.2).to(device)
history_small_reg, epochs_small = train_model_with_early_stopping(
    model_small_reg, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor,
    max_epochs=MAX_EPOCHS, batch_size=BATCH_SIZE, lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY, patience=PATIENCE
)

print("\n" + "=" * 70)
print("TRAINING: Medium Model (mit Dropout 30% + L2 + Early Stopping)")
print("=" * 70)
model_medium_reg = RegularizedMediumNet(input_dim, dropout_rate=0.3).to(device)
history_medium_reg, epochs_medium = train_model_with_early_stopping(
    model_medium_reg, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor,
    max_epochs=MAX_EPOCHS, batch_size=BATCH_SIZE, lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY, patience=PATIENCE
)

print("\n" + "=" * 70)
print("TRAINING: Large Model (mit Dropout 40% + L2 + Early Stopping)")
print("=" * 70)
model_large_reg = RegularizedLargeNet(input_dim, dropout_rate=0.4).to(device)
history_large_reg, epochs_large = train_model_with_early_stopping(
    model_large_reg, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor,
    max_epochs=MAX_EPOCHS, batch_size=BATCH_SIZE, lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY, patience=PATIENCE
)

print("\n" + "=" * 70)
print("ALLE MODELLE TRAINIERT!")
print("=" * 70)
print(f"Small Model:  Gestoppt nach {epochs_small} Epochen")
print(f"Medium Model: Gestoppt nach {epochs_medium} Epochen")
print(f"Large Model:  Gestoppt nach {epochs_large} Epochen")

TRAINING: Small Model (mit Dropout 20% + L2 + Early Stopping)


TypeError: ReduceLROnPlateau.__init__() got an unexpected keyword argument 'verbose'

## 6. Visualisierung: Loss Curves mit Early Stopping

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

histories = [
    (history_small_reg, epochs_small, 'Small Model (Reg)', 'blue'),
    (history_medium_reg, epochs_medium, 'Medium Model (Reg)', 'green'),
    (history_large_reg, epochs_large, 'Large Model (Reg)', 'red')
]

for idx, (history, final_epoch, title, color) in enumerate(histories):
    ax = axes[idx]
    
    epochs_range = range(1, len(history['train_loss']) + 1)
    
    ax.plot(epochs_range, history['train_loss'], label='Training Loss', 
            color=color, alpha=0.8, linewidth=2)
    ax.plot(epochs_range, history['val_loss'], label='Validation Loss', 
            color=color, linestyle='--', linewidth=2.5)
    
    # Markiere Stopping Point
    ax.axvline(x=final_epoch, color='red', linestyle=':', linewidth=2, alpha=0.7,
               label=f'Stopped at {final_epoch}')
    
    ax.set_xlabel('Epoch', fontsize=12)
    ax.set_ylabel('Loss (MSE)', fontsize=12)
    ax.set_title(f'{title}\nEarly Stopped', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Finale Werte
    final_train_loss = history['train_loss'][-1]
    final_val_loss = history['val_loss'][-1]
    best_val_loss = min(history['val_loss'])
    
    ax.text(0.98, 0.98, 
            f'Final Train: {final_train_loss:.4f}\nFinal Val: {final_val_loss:.4f}\nBest Val: {best_val_loss:.4f}',
            transform=ax.transAxes,
            verticalalignment='top',
            horizontalalignment='right',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.7),
            fontsize=9)

plt.tight_layout()
plt.show()

## 7. Performance-Analyse: Regularisierte Modelle

In [ ]:
models_reg = {
    'Small (Reg)': model_small_reg,
    'Medium (Reg)': model_medium_reg,
    'Large (Reg)': model_large_reg
}

results_reg = []

for name, model in models_reg.items():
    model.eval()
    with torch.no_grad():
        y_train_pred_scaled = model(X_train_tensor).squeeze().cpu().numpy()
        y_test_pred_scaled = model(X_test_tensor).squeeze().cpu().numpy()
    
    y_train_pred = y_scaler.inverse_transform(y_train_pred_scaled.reshape(-1, 1)).flatten()
    y_test_pred = y_scaler.inverse_transform(y_test_pred_scaled.reshape(-1, 1)).flatten()
    
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    
    rmse_gap = test_rmse - train_rmse
    rmse_gap_percent = (rmse_gap / train_rmse) * 100
    
    results_reg.append({
        'Model': name,
        'Train RMSE': train_rmse,
        'Test RMSE': test_rmse,
        'RMSE Gap': rmse_gap,
        'Gap %': rmse_gap_percent,
        'Train MAE': train_mae,
        'Test MAE': test_mae,
        'Train R²': train_r2,
        'Test R²': test_r2
    })

results_reg_df = pd.DataFrame(results_reg)

print("\n" + "=" * 100)
print("PERFORMANCE: Regularisierte Modelle (Dropout + L2 + Early Stopping)")
print("=" * 100)
print(results_reg_df.to_string(index=False))
print("=" * 100)

## 8. Overfitting-Analyse: Regularisiert vs. Original

Vergleichen wir die Overfitting Gaps!

In [ ]:
def analyze_overfitting_v2(train_rmse, test_rmse, model_name):
    gap = test_rmse - train_rmse
    gap_percent = (gap / train_rmse) * 100
    
    print(f"\n{'='*70}")
    print(f"ANALYSE: {model_name}")
    print(f"{'='*70}")
    print(f"Training RMSE:    ${train_rmse:>10,.2f}")
    print(f"Test RMSE:        ${test_rmse:>10,.2f}")
    print(f"Gap:              ${gap:>10,.2f} ({gap_percent:>6.1f}%)")
    print(f"{'-'*70}")
    
    if gap_percent > 30:
        verdict = "🔴 STARKES OVERFITTING"
        status = "Kritisch"
    elif gap_percent > 15:
        verdict = "🟡 MODERATES OVERFITTING"
        status = "Verbesserungswürdig"
    elif gap_percent > -5:
        verdict = "🟢 GUTER FIT"
        status = "Optimal!"
    else:
        verdict = "🔵 UNDERFITTING"
        status = "Zu einfach"
    
    median_price = df_small['median_house_value'].median()
    relative_error = (test_rmse / median_price) * 100
    
    print(f"BEWERTUNG: {verdict} ({status})")
    print(f"Relativer Test-Fehler: {relative_error:.1f}% vom Median-Preis")
    print(f"{'='*70}")

for _, row in results_reg_df.iterrows():
    analyze_overfitting_v2(row['Train RMSE'], row['Test RMSE'], row['Model'])

## 9. Visualisierung: RMSE & Overfitting Gap Vergleich

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# RMSE Vergleich
x = np.arange(len(results_reg_df))
width = 0.35

bars1 = ax1.bar(x - width/2, results_reg_df['Train RMSE'], width, 
                label='Train RMSE', alpha=0.8, color='steelblue')
bars2 = ax1.bar(x + width/2, results_reg_df['Test RMSE'], width, 
                label='Test RMSE', alpha=0.8, color='orange')

ax1.set_ylabel('RMSE ($)', fontsize=12)
ax1.set_title('RMSE Vergleich: Regularisierte Modelle\n(Dropout + L2 + Early Stopping)', 
              fontsize=14, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(results_reg_df['Model'], rotation=15)
ax1.legend()
ax1.grid(True, alpha=0.3, axis='y')

for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax1.annotate(f'${height:,.0f}',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3), textcoords="offset points",
                    ha='center', va='bottom', fontsize=9)

# Overfitting Gap
colors = ['green' if gap < 15 else 'orange' if gap < 30 else 'red' 
          for gap in results_reg_df['Gap %']]

bars = ax2.bar(results_reg_df['Model'], results_reg_df['Gap %'], 
               alpha=0.8, color=colors, edgecolor='black', linewidth=1.5)
ax2.set_ylabel('Overfitting Gap (%)', fontsize=12)
ax2.set_title('Overfitting Gap - Regularisierte Modelle\n(Niedriger = Besser)', 
              fontsize=14, fontweight='bold')
ax2.axhline(y=15, color='orange', linestyle='--', alpha=0.5, label='Moderate Grenze (15%)')
ax2.axhline(y=30, color='red', linestyle='--', alpha=0.5, label='Kritische Grenze (30%)')
ax2.set_xticklabels(results_reg_df['Model'], rotation=15)
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

for bar in bars:
    height = bar.get_height()
    ax2.annotate(f'{height:.1f}%',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3), textcoords="offset points",
                ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

## 10. FAZIT: Regularisierung vs. Ohne Regularisierung

Zusammenfassung der Verbesserungen

In [ ]:
print("\n" + "="*80)
print("FAZIT: REGULARISIERUNG WIRKT!")
print("="*80)
print("\nMit nur 80 Trainingssamples zeigen die regularisierten Modelle:")
print()

for _, row in results_reg_df.iterrows():
    if row['Gap %'] > 30:
        status = "🔴 Starkes Overfitting"
    elif row['Gap %'] > 15:
        status = "🟡 Moderates Overfitting"
    else:
        status = "🟢 Guter Fit"
    
    print(f"{row['Model']:15s} -> {status:30s} Gap: {row['Gap %']:6.1f}%")

print()
print("=" * 80)
print("ANGEWANDTE TECHNIKEN:")
print("=" * 80)
print("✓ Dropout Regularization (20-40% je nach Modellgröße)")
print("✓ L2 Weight Decay (0.01)")
print("✓ Early Stopping (Patience: 20 Epochen)")
print("✓ Learning Rate Scheduling (ReduceLROnPlateau)")
print()
print("RESULTAT:")
print(f"• Durchschnittlicher Overfitting Gap: {results_reg_df['Gap %'].mean():.1f}%")
print(f"• Bestes Modell: {results_reg_df.loc[results_reg_df['Gap %'].idxmin(), 'Model']}")
print(f"  mit Gap von {results_reg_df['Gap %'].min():.1f}%")
print()
print("Durch Regularisierung wurde das Overfitting deutlich reduziert!")
print("Die Modelle generalisieren jetzt besser auf neue Daten.")
print("=" * 80)

## 11. Predictions vs. Actual - Regularisierte Modelle

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

for idx, (name, model) in enumerate(models_reg.items()):
    model.eval()
    with torch.no_grad():
        y_train_pred_scaled = model(X_train_tensor).squeeze().cpu().numpy()
        y_test_pred_scaled = model(X_test_tensor).squeeze().cpu().numpy()
    
    y_train_pred = y_scaler.inverse_transform(y_train_pred_scaled.reshape(-1, 1)).flatten()
    y_test_pred = y_scaler.inverse_transform(y_test_pred_scaled.reshape(-1, 1)).flatten()
    
    # Training Set
    ax_train = axes[0, idx]
    ax_train.scatter(y_train, y_train_pred, alpha=0.6, s=100, edgecolors='black', color='steelblue')
    ax_train.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 
                  'r--', linewidth=2, label='Perfect Fit')
    ax_train.set_xlabel('Actual Price', fontsize=12)
    ax_train.set_ylabel('Predicted Price', fontsize=12)
    ax_train.set_title(f'{name}\nTRAINING SET (n={len(y_train)})', fontsize=12, fontweight='bold')
    ax_train.legend()
    ax_train.grid(True, alpha=0.3)
    
    train_r2 = r2_score(y_train, y_train_pred)
    ax_train.text(0.05, 0.95, f'R² = {train_r2:.3f}',
                  transform=ax_train.transAxes, verticalalignment='top',
                  bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8),
                  fontsize=11)
    
    # Test Set
    ax_test = axes[1, idx]
    ax_test.scatter(y_test, y_test_pred, alpha=0.6, s=100, 
                    edgecolors='black', color='orange')
    ax_test.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
                 'r--', linewidth=2, label='Perfect Fit')
    ax_test.set_xlabel('Actual Price', fontsize=12)
    ax_test.set_ylabel('Predicted Price', fontsize=12)
    ax_test.set_title(f'{name}\nTEST SET (n={len(y_test)})', fontsize=12, fontweight='bold')
    ax_test.legend()
    ax_test.grid(True, alpha=0.3)
    
    test_r2 = r2_score(y_test, y_test_pred)
    ax_test.text(0.05, 0.95, f'R² = {test_r2:.3f}',
                 transform=ax_test.transAxes, verticalalignment='top',
                 bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8),
                 fontsize=11)

plt.tight_layout()
plt.show()

print("Die Scatter Plots zeigen, wie gut die regularisierten Modelle die Preise vorhersagen.")
print("Punkte nahe der roten Linie = gute Predictions!")

## 12. Learning Rate Verlauf

Schauen wir uns an, wie die Learning Rate angepasst wurde.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 4))

histories_lr = [
    (history_small_reg, 'Small Model', 'blue'),
    (history_medium_reg, 'Medium Model', 'green'),
    (history_large_reg, 'Large Model', 'red')
]

for idx, (history, title, color) in enumerate(histories_lr):
    ax = axes[idx]
    epochs_range = range(1, len(history['lr']) + 1)
    
    ax.plot(epochs_range, history['lr'], color=color, linewidth=2)
    ax.set_xlabel('Epoch', fontsize=12)
    ax.set_ylabel('Learning Rate', fontsize=12)
    ax.set_title(f'{title}\nLearning Rate Schedule', fontsize=12, fontweight='bold')
    ax.set_yscale('log')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Die Learning Rate wurde automatisch reduziert, wenn kein Fortschritt erzielt wurde.")
print("Dies hilft dem Modell, feinere Anpassungen vorzunehmen.")